<a href="https://colab.research.google.com/github/taeyang916/Low-Rank-Adaptation-Practice/blob/main/Object_Detection/detr_LoRA_coco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install accelerate peft ipywidgets datasets evaluate albumentations timm transformers[torch]

In [2]:
import transformers
import accelerate
import peft
print(f"Transformers version: {transformers.__version__}")
print(f"Accelerate version: {accelerate.__version__}")
print(f"PEFT version: {peft.__version__}")

Transformers version: 4.40.1
Accelerate version: 0.31.0
PEFT version: 0.11.1


In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
model_checkpoint = "facebook/detr-resnet-50-dc5"

In [5]:
from datasets import load_dataset

dataset = load_dataset("detection-datasets/coco")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/40 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/20 [00:00<?, ?it/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 117266
    })
    val: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'objects'],
        num_rows: 4952
    })
})

In [7]:
ds = dataset['train']

In [8]:
train_dataset = dataset["train"]
val_dataset = dataset["val"]

In [54]:
ds[0]

{'image_id': 9,
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>,
 'width': 640,
 'height': 480,
 'objects': {'bbox_id': [1038967,
   1039564,
   1058555,
   1534147,
   1913551,
   1913746,
   1913856,
   1914001],
  'category': [45, 45, 50, 45, 49, 49, 49, 49],
  'bbox': [[1.08, 187.69, 612.6700000000001, 473.53],
   [311.73, 4.31, 631.01, 232.99],
   [249.6, 229.27, 565.84, 474.35],
   [0.0, 13.51, 434.48, 388.63],
   [376.2, 40.36, 451.75, 86.89],
   [465.78, 38.97, 523.85, 85.64],
   [385.7, 73.66, 469.71999999999997, 144.17000000000002],
   [364.05, 2.49, 458.81, 73.55999999999999]],
  'area': [120057.13925,
   44434.751099999994,
   49577.94434999999,
   24292.781700000007,
   2239.2924,
   1658.8913000000007,
   3609.3030499999995,
   2975.276]}}

In [9]:
ds.features['objects'].feature['category']

ClassLabel(names=['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'], id=None)

In [10]:
labels = ds.features['objects'].feature['category'].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label]=i
    id2label[i]=label

print(f"id2label[3] : {id2label[3]}")

id2label[3] : motorcycle


In [11]:
from transformers import AutoImageProcessor

image_processor = AutoImageProcessor.from_pretrained(model_checkpoint)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration. Please open a PR/issue to update `preprocessor_config.json` to use `image_processor_type` instead of `feature_extractor_type`. This warning will be removed in v4.40.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [83]:
import albumentations
import numpy as np
import torch

transform = albumentations.Compose(
    [
        albumentations.Resize(224, 224),
        albumentations.HorizontalFlip(p=0.1),
        albumentations.RandomBrightnessContrast(p=0.1),
    ],
    bbox_params=albumentations.BboxParams(format="coco", label_fields=["category"]),
)

In [84]:
def formatted_anns(image_id, category, area, bbox):
    annotations = []
    for i in range(0, len(category)):
        new_ann = {
            "image_id": image_id,
            "category_id": category[i],
            "isCrowd": 0,
            "area": area[i],
            "bbox": list(bbox[i]),
        }
        annotations.append(new_ann)

    return annotations

In [85]:
def transform_aug_ann(examples):
    image_ids = examples["image_id"]
    images, bboxes, area, categories = [], [], [], []
    for image, objects in zip(examples["image"], examples["objects"]):
        image = np.array(image.convert("RGB"))[:, :, ::-1]
        out = transform(image=image, bboxes=objects["bbox"], category=objects["category"])

        area.append(objects["area"])
        images.append(out["image"])
        bboxes.append(out["bboxes"])
        categories.append(out["category"])

    targets = [
        {"image_id": id_, "annotations": formatted_anns(id_, cat_, ar_, box_)}
        for id_, cat_, ar_, box_ in zip(image_ids, categories, area, bboxes)
    ]

    return image_processor(images=images, annotations=targets, return_tensors="pt")

In [86]:
train_dataset_transformed = train_dataset.with_transform(transform_aug_ann)
val_dataset_transformed = val_dataset.with_transform(transform_aug_ann)

In [87]:
def collate_fn(batch):
    print(batch)
    pixel_values = [item["pixel_values"] for item in batch]
    encoding = image_processor.pad(pixel_values, return_tensors="pt")
    labels = [item["labels"] for item in batch]
    batch = {}
    batch["pixel_values"] = encoding["pixel_values"]
    batch["pixel_mask"] = encoding["pixel_mask"]
    batch["labels"] = labels
    return batch

In [88]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param:.2f}"
    )

In [89]:
from transformers import AutoModelForObjectDetection
import timm

model = AutoModelForObjectDetection.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
    ignore_mismatched_sizes=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at facebook/detr-resnet-50-dc5 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForObjectDetection from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are 

In [90]:
print_trainable_parameters(model)

trainable params: 41299541 || all params: 41521941 || trainable%: 99.46


In [76]:
# check difference with ViT and DETR
vit_checkpoint = "google/vit-base-patch16-224-in21k"
vit_processor = AutoImageProcessor.from_pretrained(vit_checkpoint)
from transformers import AutoModelForImageClassification, TrainingArguments, Trainer

vit_model = AutoModelForImageClassification.from_pretrained(
    vit_checkpoint,
    label2id=label2id,
    id2label=id2label,
    ignore_mismatched_sizes=True,
)
vit_model

Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224-in21k and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ViTForImageClassification(
  (vit): ViTModel(
    (embeddings): ViTEmbeddings(
      (patch_embeddings): ViTPatchEmbeddings(
        (projection): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): ViTEncoder(
      (layer): ModuleList(
        (0-11): 12 x ViTLayer(
          (attention): ViTAttention(
            (attention): ViTSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): ViTSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): ViTIntermediate(
            (dense): Linear(in_features=7

In [77]:
model

DetrForObjectDetection(
  (model): DetrModel(
    (backbone): DetrConvModel(
      (conv_encoder): DetrConvEncoder(
        (model): FeatureListNet(
          (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
          (bn1): DetrFrozenBatchNorm2d()
          (act1): ReLU(inplace=True)
          (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
          (layer1): Sequential(
            (0): Bottleneck(
              (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (bn1): DetrFrozenBatchNorm2d()
              (act1): ReLU(inplace=True)
              (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
              (bn2): DetrFrozenBatchNorm2d()
              (drop_block): Identity()
              (act2): ReLU(inplace=True)
              (aa): Identity()
              (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      

In [91]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=['q_proj','v_proj'],
    lora_dropout=0.1,
    bias='none',
    modules_to_save=['object_detection'],
)
lora_model = get_peft_model(model, config)
print_trainable_parameters(lora_model)

trainable params: 294912 || all params: 41816853 || trainable%: 0.71


In [92]:
model_name = model_checkpoint.split('/')[-1]
batch_size = 128
model_name

'detr-resnet-50-dc5'

In [93]:
training_args = TrainingArguments(
    f"{model_name}-finetuned-lora-coco",
    remove_unused_columns=False,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    learning_rate=1e-5,
    weight_decay=1e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    max_steps=1000,
    fp16=True,
    num_train_epochs=5,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
    save_total_limit=2,
    push_to_hub=True,
    label_names=['labels'],
)

In [94]:
import numpy as np
import evaluate

metric = evaluate.load('accuracy')

def compute_metrics(eval_pred):
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [95]:
trainer = Trainer(
    lora_model,
    training_args,
    train_dataset=train_dataset_transformed,
    eval_dataset=val_dataset_transformed,
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=collate_fn,
)
train_results = trainer.train()

max_steps is given, it will override any value given in num_train_epochs


ValueError: Expected y_max for bbox (0.246203125, 0.6735128805620608, 0.559484375, 1.3792740046838405, 67) to be in the range [0.0, 1.0], got 1.3792740046838405.